<a href="https://colab.research.google.com/github/devyoungjin/VAE/blob/main/AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Flatten, Dense
from tensorflow.keras import backend as K

In [11]:
class Autoencoder:
  """
  Autoencoder represents a Deep Convolutional autoencoder architecture with mirrored encoder and decoder components.
  """
  def __init__(self,
               input_shape,
               conv_filters,
               conv_kernels,
               conv_strides,
               latent_space_dim):
    
    self.input_shape = input_shape # [28, 28, 1]
    self.conv_filters = conv_filters # [2, 4, 8]
    self.conv_kernels = conv_kernels # [3, 5, 3]
    self.conv_strides = conv_strides # [1, 2, 2]
    self.latent_space_dim = latent_space_dim # 2

    self.encoder = None
    self.decoder = None
    self.mode = None

    self._num_conv_layers = len(conv_filters)
    self._shape_before_bottleneck = None

    self._build()

  def summary(self):
    self.encoder.summary() 

  def _build(self):
    self._build_encoder()
    #self._build_dncoder()
    #self._build_autoencoder()

  def _build_encoder(self):
    encoder_input = self._add_encoder_input()
    conv_layers = self._add_conv_layers(encoder_input)
    bottleneck = self._add_bottleneck(conv_layers)
    self.encoder = Model(encoder_input, bottleneck, name="encoder")

  def _add_encoder_input(self):
    return Input(shape=self.input_shape, name="encoder_input")

  def _add_conv_layers(self,encoder_input):
    """Creates all convolutionals blocks in encoder"""
    x = encoder_input
    for layer_index in range(self._num_conv_layers):
      x = self._add_conv_layer(layer_index, x)
    return x

  def _add_conv_layer(self, layer_index, x):
    """Adds a convolutional block to a graph of layers, consisting of conv 2d + ReLU + batch normalization."""
    layer_number = layer_index + 1
    conv_layer = Conv2D(
        filters = self.conv_filters[layer_index],
        kernel_size = self.conv_kernels[layer_index],
        strides = self.conv_strides[layer_index],
        padding = "same",
        name= f"encoder_conv_layer_{layer_number}"
    )

    x = conv_layer(x)
    x = ReLU(name=f"encoder_relu_{layer_number}")(x)
    x = BatchNormalization(name=f"encoder_bn_{layer_number}")(x)

    return x 

  def _add_bottleneck(self, x):
    """ Flatten data and add bottleneck (Dense layer). """
    
    self._shape_before_bottleneck = K.int_shape(x)[1:] #[2,7,7, 32] batchsize, width, height, number of channel 인데 batchsize는 관심없음
    x = Flatten()(x)
    x = Dense(self.latent_space_dim, name="encoder_output")(x)
    return x

In [12]:
if __name__ == "__main__":
  autoencoder = Autoencoder(
      input_shape = (28,28,1),
      conv_filters = (32,64,64,64),
      conv_kernels=(3,3,3,3),
      conv_strides = (1,2,2,1),
      latent_space_dim=2
  )
  autoencoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_layer_1 (Conv2  (None, 28, 28, 32)       320       
 D)                                                              
                                                                 
 encoder_relu_1 (ReLU)       (None, 28, 28, 32)        0         
                                                                 
 encoder_bn_1 (BatchNormaliz  (None, 28, 28, 32)       128       
 ation)                                                          
                                                                 
 encoder_conv_layer_2 (Conv2  (None, 14, 14, 64)       18496     
 D)                                                              
                                                           